In [1]:
import warnings
warnings.filterwarnings("ignore")

from sqlalchemy import Boolean, Column, Integer, String

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from dotenv import load_dotenv
import os

In [2]:
load_dotenv('../config/.env',override=True)

True

In [3]:
def loadenv():
    user = os.getenv("SNOWFLAKE_USER")
    password = os.getenv("SNOWFLAKE_PASSWORD")
    db = os.getenv("SNOWFLAKE_DATABASE")
    account_identifier = os.getenv("SNOWFLAKE_ACCOUNT_IDENTIFIER")
    wh = os.getenv("SNOWFLAKE_WAREHOUSE")
    return user,password ,db ,account_identifier,wh

In [4]:
user , password, db, account_identifier, wh = loadenv()

In [5]:
def connectionToSnow(path='../config/.env',connection_test=False):
    load_dotenv(path,override=True)
    user, password, _, account_identifier,_ = loadenv()
    engine = create_engine(
        'snowflake://{user}:{password}@{account_identifier}/'.format(
            user=user,
            password=password,
            account_identifier=account_identifier,
        )
    )
    try:
        connection = engine.connect()
        results = connection.execute('select current_version()').fetchone()
        print(results[0])
        if connection_test:
            connection.close()
        else:
            return connection
    finally:
        engine.dispose()
    

In [6]:
connection = connectionToSnow()

8.5.2


In [7]:
def execute(connection,query):
    try:
        results = connection.execute(query)
    except Exception as e:
        print("error-->",e)
    finally:
        print("Done")
        
def execute_query(connection,query):
    try:
        results = connection.execute(query)
    except Exception as e:
        print("error-->",e)
    finally:
        print("Done")

## execute statements 

In [9]:

drop_database_query = '''DROP DATABASE IF EXISTS {};'''.format(db)

create_database_query = "CREATE OR REPLACE DATABASE {};".format(db)
create_warehouse = """CREATE OR REPLACE WAREHOUSE {} WITH
   WAREHOUSE_SIZE='X-SMALL'
   AUTO_SUSPEND = 180
   AUTO_RESUME = TRUE
   INITIALLY_SUSPENDED=TRUE;
   """.format(wh)
drop_web_data_table = '''DROP TABLE IF EXISTS {};'''.format('CFA_WEB_DATA_R')
create_web_data_table = '''CREATE TABLE {} (
    pdf_link VARCHAR,
    Parent_topic VARCHAR,
    year VARCHAR,
    level VARCHAR,
    Introduction VARCHAR,
    LearningOutcome VARCHAR,
    Summary VARCHAR,
    categories VARCHAR, 
    url VARCHAR,
    topicName VARCHAR
);'''.format('CFA_WEB_DATA_R')

current_directory = os.getcwd()
# Get the parent directory
parent_directory = os.path.dirname(current_directory)

new_file_path = parent_directory + "/sample_output/"


create_stage = """CREATE OR REPLACE STAGE DUMMY DIRECTORY = ( ENABLE = true );"""
upload_to_stage  = """PUT file://{}*.csv @{}.public.dummy;""".format(new_file_path,db)

copy_stage_to_table = """COPY INTO {}
  FROM @{}.public.dummy
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"')
  PATTERN = 'FinanceHub.csv.gz'
  ON_ERROR = 'skip_file';""".format('CFA_WEB_DATA_R',db,)


In [ ]:


# Get the parent directory
parent_directory = os.path.dirname(current_directory)
new_file_path = parent_directory + "/sample_output/"

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)
print('file path:',new_file_path)

In [10]:
execute(connection, drop_database_query)
execute(connection, create_database_query)
execute(connection, create_warehouse)

execute(connection, drop_web_data_table)
execute(connection, create_web_data_table)


execute(connection, create_stage)
execute(connection, upload_to_stage)
execute(connection, copy_stage_to_table)

Done
Done
Done
Done
Done
Done
Done
Done
